In [ ]:
from astropy.io import ascii
from astropy.io import fits
from astropy.modeling import models, fitting
from astropy.stats import sigma_clip
from astropy.table import Table, unique
import matplotlib.pyplot as plt
import numpy as np
import os

%run Crossmatch.ipynb

# Performs the cross-match between the image catalog and the appropriate catalogs and prepares for the calibration function
def photometry_calib_prep_psf(filter_cat, filter_ra, filter_dec, filter_flux, exposure, cat_1, cat_1_ra, cat_1_dec, cat_1_flux, cat_2, cat_2_ra, cat_2_dec, cat_2_flux, arc_range, save_path, format_dat, ow, plot):
    
    # Creates the save_path folder if it does not already exist
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    
    # Creates a copy of the catalog to-be-calibrated and stores it in a new Table object
    filter_cat = Table.copy(filter_cat)
    
    # Removes sources with SExtractor flags from the above catalog
    filter_cat.remove_rows([filter_cat['FLAGS'] != 0])
    
    # Remove sources with high ellipticites from the above catalog
    filter_cat.remove_rows([filter_cat['ELLIPTICITY'] >= 0.5])
    
    # Creates a copy of the first catalog and stores it in a new Table object
    cat_1 = Table.copy(cat_1) 
    
    # Removes sources from the above catalog with a magnitude measurement associated error bigger than 0.1
    cat_1.remove_rows([cat_1['e_Imag'] >= 0.1])
    
    # Cross-matches the filter catalog with the first catalog
    cross_filter_cat_1 = cross_match(filter_cat, filter_ra, filter_dec, cat_1, cat_1_ra, cat_1_dec, arc_range, sep = True)
    
    # Resets the column names of the Table object
    cross_filter_cat_1 = reset_main_catalog_columns(filter_cat, cross_filter_cat_1)
    
    cross_filter_cat_1.sort('d2d')
    
    # Removes the duplicate entries from the cross-matched catalog
    cross_filter_cat_1 = unique(cross_filter_cat_1, keys = [cat_1_ra, cat_1_dec], keep = 'first', silent = True)
    
    # Calculates the mean flux of the filter entries in the cross-matched catalog
    mean_flux_filter = np.nanmean(cross_filter_cat_1[filter_flux])
    
    # If the flux values haven't yet been converted to magnitudes
    if mean_flux_filter < - 999 or mean_flux_filter > 999:
        
        # Creates a Column copy of the flux values column
        cross_filter_cat_1['FLUX_COUNTS'] = Column.copy(cross_filter_cat_1[filter_flux])
        
        # Converts the flux errors into magnitude errors
        cross_filter_cat_1['TEMP_MAG_ERR'] = (- 2.5 / (np.log(10) * cross_filter_cat_1['FLUX_PSF'])) * cross_filter_cat_1['FLUXERR_PSF']
    
        # Converts the flux values into magnitudes
        cross_filter_cat_1[filter_flux] = - 2.5 * np.log10(cross_filter_cat_1[filter_flux] / exposure)
    
    # Creates an array with the magnitude differences between the first catalog and the filter catalog
    cross_filter_cat_1['Cat_1 - Filter'] = cross_filter_cat_1[cat_1_flux] - cross_filter_cat_1[filter_flux]
    
    # The save-path to where the first cross-matched catalog will be stored
    cross_filter_cat_1_path = save_path + 'Cross_Filter_Cat_1.dat'
    
    # Saves the first cross-matched catalog
    ascii.write(cross_filter_cat_1, cross_filter_cat_1_path, format = format_dat, overwrite = ow)
    
    # Cross-matches the filter catalog with the second catalog
    cross_filter_cat_2 = cross_match(cross_filter_cat_1, filter_ra, filter_dec, cat_2, cat_2_ra, cat_2_dec, arc_range, sep = True)
    
    # Resets the column names of the Table object
    cross_filter_cat_2 = reset_main_catalog_columns(filter_cat, cross_filter_cat_2)
    
    cross_filter_cat_1.sort('d2d')
    
    # Removes the duplicate entries from the cross-matched catalog
    # During the CROSS_MATCH function, if both catalogs have the same column names for the right ascenscion and
    # declination columns, during HSTACK these columns are renamed to the original name plus "_1" and "_2"
    # This exception accounts for that
    try:
        cross_filter_cat_2 = unique(cross_filter_cat_2, keys = [cat_2_ra, cat_2_dec], keep = 'first', silent = True)
    
    except KeyError:
        
        cat_2_ra = cat_2_ra + '_2'
        cat_2_dec = cat_2_dec + '_2'
        cross_filter_cat_2 = unique(cross_filter_cat_2, keys = [cat_2_ra, cat_2_dec], keep = 'first', silent = True)
    
    # VHSDR6 sets default extremely high values for objects with no magnitude measurements
    # These objects with no magnitude measurements are removed in these two instructions
    cross_filter_cat_2.remove_rows([cross_filter_cat_2[cat_2_flux] > 999])
    cross_filter_cat_2.remove_rows([cross_filter_cat_2[cat_2_flux] < - 999])
    
    # Creates an array with the magnitude differences between the second catalog and the filter catalog
    cross_filter_cat_2['Cat_2 - Filter'] = cross_filter_cat_2[cat_2_flux] - cross_filter_cat_2[filter_flux]
    
    # Creates an array with the magnitude differences between the first catalog and the second catalog
    cross_filter_cat_2['Cat_1 - Cat_2'] = cross_filter_cat_2[cat_1_flux] - cross_filter_cat_2[cat_2_flux]
    
    # The savepath to where the second cross-matched catalog will be stored
    cross_filter_cat_2_path = save_path + 'Cross_Filter_Cat_2.dat'
    
    # Saves the second cross-matched catalog
    ascii.write(cross_filter_cat_2, cross_filter_cat_2_path, format = format_dat, overwrite = ow)
    
    # Plots the difference vector between the catalog flux values with the filter flux values versus the difference
    # vector between the first and second catalog flux values if the user has requested so
    if plot == "Yes":
        
        calibration_and_plot(cross_filter_cat_2_path, 'Cat_1 - Cat_2', 'Cat_1 - Filter', 2)
        
        calibration_and_plot(cross_filter_cat_2_path, 'Cat_1 - Cat_2', 'Cat_2 - Filter', 2)

# Returns the necessary values to perform photometric calibration, as well as plot the color-color diagram
def calibration_and_plot(cross_path, x_axis_1, y_axis_1, k, min_mag, max_mag, color_cat, color_inst, min_mag_denis = -999, max_mag_denis = 999):
    
    # Reads the catalog file into a Table object
    cross = ascii.read(cross_path)
    
    # Initializes an np.array() of the CROSS Table object column which will correspond to the X_AXIS and Y_AXIS in the catalog magnitude vs. instrumental magnitude plot
    # These entries will be used for the calculation of the zero-point and color-term corrections
    arr_x_1 = np.array(cross[x_axis_1])
    arr_y_1 = np.array(cross[y_axis_1])
    
    # Initializes an np.array() of the CROSS Table object column which will correspond to the catalog color in the color-color plots
    # These entries will be used for the calculation of the zero-point and color-term corrections
    arr_color_cat = np.array(cross[color_cat])
    
    # Initializes an np.array() of the CROSS Table object column which will correspond to the instrumental color in the color-color plots
    # These entries will be used for the calculation of the zero-point and color-term corrections
    arr_color_inst = np.array(cross[color_inst])
    
    # Preserves only the calibration catalog magnitudes above a certain limit
    arr_y_1 = arr_y_1[arr_x_1 > min_mag]
    arr_color_cat = arr_color_cat[arr_x_1 > min_mag]
    arr_color_inst = arr_color_inst[arr_x_1 > min_mag]
    arr_x_1 = arr_x_1[arr_x_1 > min_mag]
    # Preserves only the calibration catalog magnitudes below a certain limit
    arr_y_1 = arr_y_1[arr_x_1 < max_mag]
    arr_color_cat = arr_color_cat[arr_x_1 < max_mag]
    arr_color_inst = arr_color_inst[arr_x_1 < max_mag]
    arr_x_1 = arr_x_1[arr_x_1 < max_mag]
    
    # Initializes the outlier removal fitter
    or_fit = fitting.FittingWithOutlierRemoval(fitting.LinearLSQFitter(), sigma_clip, niter = 3, sigma = k)
    
    # Fit the data to the above fitter
    fitted_line, mask = or_fit(models.Linear1D(), arr_x_1, arr_y_1)
    arr_x_1 = np.ma.masked_array(arr_x_1, mask = mask)
    arr_y_1 = np.ma.masked_array(arr_y_1, mask = mask)
    arr_color_cat = np.ma.masked_array(arr_color_cat, mask = mask)
    arr_color_inst = np.ma.masked_array(arr_color_inst, mask = mask)
    
    # This will serve to highlight which matches are being used to calculate the corrections
    
    # All values in the column equals to zero will be transformed into NaN
    # This happens because some column cells have empty values which are converted to zero by the np.array() method
    arr_color_inst[arr_color_inst == 0.0] = np.nan
    arr_color_cat = arr_color_cat[np.isfinite(arr_color_inst)]
    arr_color_inst = arr_color_inst[np.isfinite(arr_color_inst)]
    
    # All values in the column equals to zero will be transformed into NaN
    # This happens because some column cells have empty values which are converted to zero by the np.array() method
    arr_color_cat[arr_color_cat == 0.0] = np.nan
    arr_color_inst = arr_color_inst[np.isfinite(arr_color_cat)]
    arr_color_cat = arr_color_cat[np.isfinite(arr_color_cat)]
    
    # Initializes np.array()s of the CROSS Table object columns which will correspond to the X_AXIS and Y_AXIS in the plots
    # These entries correspond to all the cross-matches present in the CROSS Table object
    arr_x_1_total = np.array(cross[x_axis_1])
    arr_y_1_total = np.array(cross[y_axis_1])
    
    # Initializes np.array()s of the CROSS Table object columns which will correspond to the X_AXIS and Y_AXIS in the plots
    # These entries correspond to all the cross-matches present in the CROSS Table object
    arr_color_cat_total = np.array(cross[color_cat])
    arr_color_inst_total = np.array(cross[color_inst])
    
    # All values in the column equals to zero will be transformed into NaN
    # This happens because some column cells have empty values which are converted to zero by the np.array() method
    arr_y_1_total[arr_y_1_total == 0.0] = np.nan
    arr_x_1_total = arr_x_1_total[np.isfinite(arr_y_1_total)]
    arr_color_cat_total = arr_color_cat_total[np.isfinite(arr_y_1_total)]
    arr_color_inst_total = arr_color_inst_total[np.isfinite(arr_y_1_total)]
    arr_y_1_total = arr_y_1_total[np.isfinite(arr_y_1_total)]
    
    # All values in the column equals to zero will be transformed into NaN
    # This happens because some column cells have empty values which are converted to zero by the np.array() method
    arr_x_1_total[arr_x_1_total == 0.0] = np.nan
    arr_y_1_total = arr_y_1_total[np.isfinite(arr_x_1_total)]
    arr_color_cat_total = arr_color_cat_total[np.isfinite(arr_x_1_total)]
    arr_color_inst_total = arr_color_inst_total[np.isfinite(arr_x_1_total)]
    arr_x_1_total = arr_x_1_total[np.isfinite(arr_x_1_total)]
    
    # All values in the column equals to zero will be transformed into NaN
    # This happens because some column cells have empty values which are converted to zero by the np.array() method
    arr_color_cat_total[arr_color_cat_total == 0.0] = np.nan
    arr_color_inst_total = arr_color_inst_total[np.isfinite(arr_color_cat_total)]
    arr_color_cat_total = arr_color_cat_total[np.isfinite(arr_color_cat_total)]
    
    # All values in the column equals to zero will be transformed into NaN
    # This happens because some column cells have empty values which are converted to zero by the np.array() method
    arr_color_inst_total[arr_color_inst_total == 0.0] = np.nan
    arr_color_cat_total = arr_color_cat_total[np.isfinite(arr_color_inst_total)]
    arr_color_inst_total = arr_color_inst_total[np.isfinite(arr_color_inst_total)]
    
    if min_mag_denis != -999:
        # Preserves only the calibration catalog magnitudes below a certain limit
        arr_x_1 = arr_x_1[arr_y_1 > min_mag_denis]
        arr_color_cat = arr_color_cat[arr_y_1 > min_mag_denis]
        arr_color_inst = arr_color_inst[arr_y_1 > min_mag_denis]
        arr_y_1 = arr_y_1[arr_y_1 > min_mag_denis]
    
    if max_mag_denis != 999:
        # Preserves only the calibration catalog magnitudes below a certain limit
        arr_x_1 = arr_x_1[arr_y_1 < max_mag_denis]
        arr_color_cat = arr_color_cat[arr_y_1 < max_mag_denis]
        arr_color_inst = arr_color_inst[arr_y_1 < max_mag_denis]
        arr_y_1 = arr_y_1[arr_y_1 < max_mag_denis]
    
    # Creates a linear-fit using the data of the two arrays where C_T is the
    # slope value (which, in this case, corresponds to the color-term) and Z_P is the zero-point value
    m_b, cov = np.polyfit(arr_color_cat, arr_color_inst, 1, cov = True)
    
    # Creates a zero-order fit using the data to calculate the calibration without a color-term correction
    m_0, cov_0 = np.polyfit(arr_color_cat, arr_color_inst, 0, cov = True)
    
    print("Zero-Point-only-correction is:", m_0[0], "+/-", cov_0[0][0])
    
    # For plotting the errors in the I vs. I figure
    
    subaru_mean_err = np.nanmean(cross['TEMP_MAG_ERR'])
    denis_mean_err = np.nanmean(cross['e_Imag'])
    
    color_inst_mean_err = np.nanmean(cross['TEMP_MAG_ERR'] + cross['e_Imag'])
    color_cat_mean_err = np.nanmean(cross['e_Imag'] + cross['jAperMag3Err'])
    
    plt.rcParams.update(plt.rcParamsDefault)
    
    plt.rc('xtick',labelsize = 16)
    plt.rc('ytick',labelsize = 16)
    
    # Plots the entire arrays with the sub-array used to perform the linear-fit overplotted on top in ORANGE
    # This is a magnitude-magnitude diagram between the filter catalog and the calibration catalog
    plt.figure(1, figsize = (5, 5))
    plt.scatter(arr_x_1_total, arr_y_1_total, s = 10, label = 'I-band Sources')
    plt.scatter(arr_x_1, arr_y_1, s = 10, label = 'Selec. Sources')
    #plt.xlim(-11.5, -9)
    plt.ylim(14.9, 17.5)
    x_err_bar_x = [plt.xlim()[0] + 0.2, plt.xlim()[0] + 0.2 + 100 * subaru_mean_err]
    x_err_bar_y = [plt.ylim()[0] + 0.1 + denis_mean_err / 2, plt.ylim()[0] + 0.1 + denis_mean_err / 2]
    y_err_bar_x = [plt.xlim()[0] + 0.2 + 100 * subaru_mean_err / 2, plt.xlim()[0] + 0.2 + 100 * subaru_mean_err / 2]
    y_err_bar_y = [plt.ylim()[0] + 0.1 + denis_mean_err, plt.ylim()[0] + 0.1]
    plt.plot(x_err_bar_x, x_err_bar_y, color = 'b')
    plt.plot(y_err_bar_x, y_err_bar_y, color = 'b')
    plt.text(plt.xlim()[0], plt.ylim()[1], cross['DETECTOR'][0], color = 'k', fontsize = 20)
    plt.xlabel(r'I$_{SUBARU}$ [mag]', fontsize = 'xx-large')
    plt.ylabel(r'I$_{DENIS}$ [mag]', fontsize = 'xx-large')
    plt.xlabel(r'I$_{SUBARU}$ [mag]', fontsize = 'xx-large')
    plt.ylabel(r'I$_{DENIS}$ [mag]', fontsize = 'xx-large')
    #if 'W-S-I+_clarisse_stacked_tmp' in cross['DETECTOR']:
        #plt.xlim(-14, -9)
    #plt.legend(fontsize = 'large', loc = 4)
    plt.show()
    
    x = np.linspace(np.amin(arr_color_cat), np.amax(arr_color_cat), 100)
    y = m_b[0] * x + m_b[1]
    
    # Plots the entire arrays with the sub-array used to perform the linear-fit overplotted on top in ORANGE
    # This is the color-color plot
    plt.figure(2, figsize = (5, 5))
    #plt.scatter(arr_color_cat_total, arr_color_inst_total, s = 10, label = 'I-band Sources')
    plt.scatter(arr_color_cat, arr_color_inst, s = 10, label = 'Selec. Sources', color = '#ff7f0e')
    plt.plot(x, y, 'k', label = 'Linear-fit')
    #plt.xlim(0.5, 4)
    plt.ylim(25, 28)
    x_err_bar_x = [plt.xlim()[0] + 0.1, plt.xlim()[0] + 0.1 + color_cat_mean_err]
    x_err_bar_y = [plt.ylim()[0] + 0.1 + color_inst_mean_err / 2, plt.ylim()[0] + 0.1 + color_inst_mean_err / 2]
    y_err_bar_x = [plt.xlim()[0] + 0.1 + color_cat_mean_err / 2, plt.xlim()[0] + 0.1 + color_cat_mean_err / 2]
    y_err_bar_y = [plt.ylim()[0] + 0.1 + color_inst_mean_err, plt.ylim()[0] + 0.1]
    plt.plot(x_err_bar_x, x_err_bar_y, color = 'b')
    plt.plot(y_err_bar_x, y_err_bar_y, color = 'b')
    plt.text(plt.xlim()[0], plt.ylim()[1], cross['DETECTOR'][0], color = 'k', fontsize = 20)
    plt.xlabel(r'I$_{DENIS}$ - J$_{VHS}$ [mag]', fontsize = 'xx-large')
    plt.ylabel(r'I$_{DENIS}$ - I$_{SUBARU}$ [mag]', fontsize = 'xx-large')
    #plt.legend(fontsize = 'xx-large', loc = 4)
    if 'W-S-I+_clarisse_stacked_tmp' in cross['DETECTOR']:
        plt.xlim(0.2, 3)
        plt.ylim(26, 26.75)
    #plt.legend(fontsize = 'large', loc = 4)
    plt.show()
    
    print("Color-Term is:", m_b[0], "+/-", np.sqrt(cov[0][0]))
    print("Zero-Point is:", m_b[1], "+/-", np.sqrt(cov[1][1]))
    
    # Returns the zero-point measured by the linear-fit for the given DETECTOR as well as its color-term and their respective uncertainties
    return m_0, cov_0, m_b, cov
